# 🎯 Strategic Customer Segmentation: From Data to Action
## A Business-Centric Data Science Case Study
---
### **Executive Summary**
In the fiercely competitive online retail space, treating every customer the same is a recipe for inefficiency. This project answers critical business questions to unlock value from transactional data.

**Primary Objective**: Segment the customer base to enable hyper-targeted marketing strategies.

**Key Questions We Answer:**
1. *Is our data quality sufficient for reliable decision-making?*
2. *How do our customers behave in terms of Recency, Frequency, and Monetary value?*
3. *Can we identify distinct personas to tailor our engagement?*

In [1]:
import sys
from pathlib import Path
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Import modular src
sys.path.append(str(Path.cwd().parent / 'src'))

from data import extract
from features import cleaning, engineering as build_features, pipeline
from models import training as train_model
from visualization import reporting

print('✅ Environment & Modules Loaded')

✅ Environment & Modules Loaded


## 🔍 Q1: Is our data trusted and ready?
**Context**: Raw transactional data often contains noise (returns, test transactions, errors). Validating data health is the first step to trustable insights.

In [2]:
# Load and Clean Data
raw_df = extract.load_raw_dataset()
print(f"📦 Raw Input: {len(raw_df):,} transactions")

clean_df = pipeline.cleaning_pipeline(raw_df)

print('✅ Executed Project Leader Cleaning Pipeline (Steps 1-9)')

# Visualizing Data Retention
retention_data = pd.DataFrame({
    'Stage': ['Raw Data', 'Clean Data'],
    'Transactions': [len(raw_df), len(clean_df)]
})

fig = px.bar(retention_data, x='Stage', y='Transactions', 
             title='Data Health Check: Retention Pipeline', 
             text='Transactions', color='Stage')
fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.show()

✅ DataExtractor inicializado:
   Proyecto: /home/datascientist/workspace/github-collabs/ds_projects_collabs/1-cluster_retail_uci
   Excel: /home/datascientist/workspace/github-collabs/ds_projects_collabs/1-cluster_retail_uci/data/raw/online_retail_II.xlsx
   Output: /home/datascientist/workspace/github-collabs/ds_projects_collabs/1-cluster_retail_uci/data/processed
📥 Cargando TODAS las hojas...
✅ Cargadas 2 hojas:
   • Year 2009-2010: 525,461 filas
   • Year 2010-2011: 541,910 filas
📦 Raw Input: 1,067,371 transactions
🚀 Starting cleaning pipeline...
✅ 'invoice' is already string!
📋 Results for prefix: 'C'


,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321,Australia


📋 Results for prefix: 'A'


,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom
825443,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom


✅ 'stockcode' is already string!
✅ Removed 4879 rows where 'stockcode' was excluded.
📋 Number of duplicated rows: 26056
✅ Duplicates dropped.
✅ Rows with price equal to 0 removed.
✅ Cleaning pipeline finished successfully.
✅ Executed Project Leader Cleaning Pipeline (Steps 1-9)


### 💡 Findings & Improvements
- **Finding**: We retained approx 73% of raw transactions. 
- **Issue Identified**: A significant portion of records lacked valid Customer IDs or were cancellations.
- **Improvement**: Investigate upstream data collection for Guest Checkout processes to capture more Customer IDs.

## 🧬 Q2: What is the behavioral DNA of our customers?
We break down behavior into three dimensions (RFM) to understand value distribution.

In [3]:
# Feature Engineering
rfm_df = build_features.create_rfm_features(clean_df)

# Interactive 3D Exploration
fig = px.scatter_3d(rfm_df, x='recency_days', y='frequency', z='sale_value',
                    opacity=0.6, title='Customer Behavioral Space (RFM)',
                    labels={'recency_days': 'Recency (Days)', 'frequency': 'Frequency', 'sale_value': 'Monetary ($)'})
fig.update_layout(scene = dict(
                    xaxis_title='Recency (Days)',
                    yaxis_title='Frequency (#)',
                    zaxis_title='Monetary Value ($)'))
fig.show()

🧬 Calculating RFM features...
   ⚠️ 'sale_total' column missing. Calculating it (quantity * price)...
   ✅ Features created for 5,852 customers.


### 💡 Findings
- **Skewed Value**: A small cluster of customers drives a disproportionate amount of revenue (High Monetary).
- **Drift**: We have a visible segment of high-frequency shoppers who haven't purchased recently (High Frequency, High Recency).

## 👥 Q3: Can we group them into actionable personas?
Using K-Means clustering on pre-processed RFM data to mathematically define personas.

In [4]:
# Preprocessing & Modelling
processed_df = build_features.preprocess_rfm(rfm_df)
kmeans, clusters = train_model.run_clustering(processed_df, k=4)
rfm_df['cluster'] = clusters

# Cluster Profiling
cluster_stats = reporting.describe_clusters(rfm_df, ['recency_days', 'frequency', 'sale_value'])
display(cluster_stats)

# Visualizing Clusters
reporting.plot_3d_clusters(rfm_df, kmeans, ['recency_days', 'frequency', 'sale_value'])

🛠️ Pre-processing RFM features (Log + Scale)...
K-Means model fitted with 4 clusters.


,recency_days,frequency,sale_value,n_customers
cluster,,,,
0,388.05,1.42,337.78,2058
1,24.22,19.98,11086.17,1100
2,26.39,3.01,832.21,1207
3,207.54,5.43,2134.61,1487


## 🚀 Strategic Recommendations
Based on the 4 identified personas, we propose the following specific actions:

| Persona | Characteristics | Strategy | Possible Improvement |
| :--- | :--- | :--- | :--- |
| **👑 VIPs** | High Spend, Frequent, Recent | **Retain & Reward** | Exclusive concierge service, early access. |
| **📈 Loyalists** | Moderate Spend, Regular | **Upsell** | Bundle offers to increase basket size. |
| **📉 At Risk** | Good history, High Recency | **Reactivate** | "We miss you" coupons, surveys to find friction. |
| **🆕 Newcomers** | Low Freq, Very Recent | **Onboard** | Content marketing, welcome series to drive 2nd purchase. |

### 🔮 Next Steps
- **Automate**: Deploy this pipeline to run weekly.
- **Enrich**: Add product category affinity to segmentation.